### Predicting Mask on Augmented Dataset

1. Load Model
2. Load Test Dataset - Images
3. Predict Mask
4. Calculate FPS

In [7]:
import os
import time
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf

#### Seeding

In [8]:
# Seeding
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)

#### Hyperparameters

In [9]:
# Hyperparameters
height = 512
width = 768
num_classes = 3

#### Path

In [10]:
# Paths
dataset_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/processed_dataset/aug/test"
save_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/predicted_masks/aug"
model_path = "/home/ahsan/University/Thesis/UNet_Directory/Datasets/second_phase/files/aug/unet-multiclass.keras"

#### Create Folder

In [11]:
os.makedirs(save_path, exist_ok=True)

#### Load Model

In [12]:
# Load Model (update to .keras format)
model = tf.keras.models.load_model(model_path, compile=False)

In [13]:
model.summary()

Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 768, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 768, 512,  │      1,792 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 768, 512,  │        256 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_90       │ (None, 768, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 768, 512,  │     36,928 │ activation_90[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 768, 512,  │        256 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_91       │ (None, 768, 512,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_20    │ (None, 384, 256,  │          0 │ activation_91[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 384, 256,  │     73,856 │ max_pooling2d_20… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 384, 256,  │        512 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_92       │ (None, 384, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 384, 256,  │    147,584 │ activation_92[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 384, 256,  │        512 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_93       │ (None, 384, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_21    │ (None, 192, 128,  │          0 │ activation_93[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_99 (Conv2D)  │ (None, 192, 128,  │    295,168 │ max_pooling2d_21… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 192, 128,  │      1,024 │ conv2d_99[0][0] 

 Total params: 31,055,427 (118.47 MB)

 Trainable params: 31,043,651 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

#### Load Test Dataset: Images

In [14]:
# Define color map for visualization (BGR format)
colors = {
    0: (0, 0, 0),       # Non-Drivable Area (black)
    1: (79, 247, 211),  # My Way (#d3f74f) in BGR
    2: (247, 93, 79)    # Other Way (#4f5df7) in BGR
}

In [15]:
# Load test images
test_x = sorted(glob(os.path.join(dataset_path, "images", "*")))

#### Predict the mask and calculate FPS

In [16]:
time_taken = []

for x_path in tqdm(test_x, desc="Predicting Masks"):
    name = os.path.basename(x_path)
    x = cv2.imread(x_path, cv2.IMREAD_COLOR)

    if x is None:
        print(f"Error loading image {x_path}, skipping.")
        continue

    x = cv2.resize(x, (width, height))
    x = x / 255.0
    x = np.expand_dims(x, axis=0)

    start_time = time.time()
    p = model.predict(x)[0]
    time_taken.append(time.time() - start_time)

    p = np.argmax(p, axis=-1).astype(np.uint8)  # Convert probabilities to class labels

    # Create an empty colored mask
    p_colored = np.zeros((height, width, 3), dtype=np.uint8)

    # Assign colors based on class labels
    for label, color in colors.items():
        p_colored[p == label] = color

    # Save predicted mask
    cv2.imwrite(os.path.join(save_path, f"{name.split('.')[0]}.png"), p_colored)


Predicting Masks:   0%|          | 0/3 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 990ms/step


Predicting Masks:  33%|███▎      | 1/3 [00:01<00:02,  1.12s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step


Predicting Masks:  67%|██████▋   | 2/3 [00:01<00:00,  1.06it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 764ms/step


Predicting Masks: 100%|██████████| 3/3 [00:02<00:00,  1.10it/s]


In [17]:
# FPS Calculation
mean_time = np.mean(time_taken)
mean_fps = 1 / mean_time

In [18]:
print(f"Mean time taken: {mean_time:.4f} seconds")
print(f"Mean FPS: {mean_fps:.2f}")

Mean time taken: 0.8603 seconds
Mean FPS: 1.16
